In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import numpy as np
from math import cos, radians
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import from_origin
from pyproj import Proj, transform
import pyproj

In [2]:
import calc_footprint_FFP_climatology as ffp
import footprint_FKM_climatology as fkm

In [26]:
df = pd.read_csv('stable.csv')

In [27]:
df.head()

,mm,day,yyyy,HH_UTC,MM,umean1,umean,wind_dir,z0,ol,u_star,sigma_v,Zm_1,Zm_2
0,3,23,2009,4,4,0.25,1.00,1,63818.000000,0.97356,0.019128,0.52,2.83,5.66
1,2,20,2009,5,4,1.25,1.95,1,0.005986,2.39770,0.041245,0.65,2.83,5.66
2,12,20,2009,17,3,0.90,1.45,1,0.128600,4.06120,0.055067,0.26,2.83,5.66
3,2,21,2009,5,5,0.50,1.40,1,10.644000,4.74020,0.097033,0.45,2.83,5.66
4,2,20,2009,3,5,0.30,1.00,1,3.838700,14.51400,0.165770,0.59,2.83,5.66


In [10]:
# df.columns

In [11]:
# df.columns

In [28]:
df1 = df[(df['mm'] == 12) | (df['mm'] < 3)]  # winter
df2 = df[(df['mm'] >=3) & (df['mm'] <= 5)]   # summer
df3 = df[(df['mm'] >= 6) & (df['mm'] <= 9)]   # monsoon
df4 = df[(df['mm'] >= 10) & (df['mm'] <= 11)]   # autumn

In [15]:
# Constants and parameters
zm_s = 5.66  # Measurement height [m] from AMF metadata
h_s = 2000.  # Height of atmospheric boundary layer [m] - assumed
h_c = np.array(df2['z0'].tolist())  # Height of atmospheric boundary layer [m]
dx = 0.5  # Model resolution [m]
origin_d = 200.  # Model bounds distance from origin [m]

# Station coordinates (lat, long)
latitude = 22.53809845
longitude = 72.98046952
station_coord = (latitude, longitude)

# Calculate the appropriate EPSG code for UTM based on latitude and longitude
EPSG = int(32700 - np.round((45 + latitude) / 90.0) * 100 + np.round((183 + longitude) / 6.0))

# Use pyproj to transform from lat/long (EPSG:4326) to UTM coordinates
transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG}", always_xy=True)
station_x, station_y = transformer.transform(longitude, latitude)

# Check the UTM coordinate results
print(f"Station coordinates in UTM: X = {station_x}, Y = {station_y}, EPSG = {EPSG}")

# Placeholder for the FFP model (assuming fkm is properly defined and used in your environment)
# Initialize FFP model
ffp = fkm.FKM_climatology(domain=[-origin_d, origin_d, -origin_d, origin_d],
                           dx=dx, dy=dx,
                           zm=df2['Zm_2'].tolist(), 
                           z0=df2['z0'].tolist(), 
                           umean=df2['umean'].tolist(),
                           ol=df2['ol'].tolist(), 
                           sigmav=df2['sigma_v'].tolist(), 
                           ustar=df2['u_star'].tolist(), 
                           wind_dir=df2['wind_dir'].tolist())

# Process the footprint data (assuming ffp returns the correct structure)
f_2d = np.array(ffp['fclim_2d']) 
x_2d = np.array(ffp['x_2d']) + station_x
y_2d = np.array(ffp['y_2d']) + station_y
f_2d = f_2d * dx ** 2 

# Create affine transformation for the GeoTIFF
affine_transform = from_origin(station_x - origin_d, station_y + origin_d, dx, dx)

# Set CRS (Coordinate Reference System) using rasterio's CRS
utm_crs = pyproj.CRS.from_epsg(EPSG).to_wkt()
# Prepare to write the GeoTIFF file
out_f = 'new_summer_5_unstable.tif'
with rasterio.open(out_f, 'w', driver='GTiff', dtype=rasterio.float64,
                   count=1,  # Only one band for this footprint
                   height=f_2d.shape[0], width=f_2d.shape[1],
                   transform=affine_transform,
                   crs=utm_crs,
                   nodata=0.0) as new_dat:
    new_dat.write(f_2d, 1)  # Write to the first band

print(f"GeoTIFF saved as {out_f}")

Station coordinates in UTM: X = 292307.3593888789, Y = 2493792.267627137, EPSG = 32643
Percent: [--------------------------------------------------] 0.0% 

d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: divide by zero encountered in divide
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: overflow encountered in exp
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: overflow encountered in multiply
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))


Percent: [##################################################] 99.0% 00000000001% 

188 times Error(0007):
 zm/ol (measurement height to Obukhov length ratio) must be equal or larger than -15.5.
7 times Error(0008):
 sigmav (standard deviation of crosswind) must be larger than zero.
21 times Error(0009):
 ustar (friction velocity) must be >=0.1.


GeoTIFF saved as new_summer_5_unstable.tif


In [29]:
# Constants and parameters
zm_s = 5.66  # Measurement height [m] from AMF metadata
h_s = 2000.  # Height of atmospheric boundary layer [m] - assumed
h_c = np.array(df4['z0'].tolist())  # Height of atmospheric boundary layer [m]
dx = 0.5  # Model resolution [m]
origin_d = 200.  # Model bounds distance from origin [m]

# Station coordinates (lat, long)
latitude = 22.53809845
longitude = 72.98046952
station_coord = (latitude, longitude)

# Calculate the appropriate EPSG code for UTM based on latitude and longitude
EPSG = int(32700 - np.round((45 + latitude) / 90.0) * 100 + np.round((183 + longitude) / 6.0))

# Use pyproj to transform from lat/long (EPSG:4326) to UTM coordinates
transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG}", always_xy=True)
station_x, station_y = transformer.transform(longitude, latitude)

# Check the UTM coordinate results
print(f"Station coordinates in UTM: X = {station_x}, Y = {station_y}, EPSG = {EPSG}")

# Placeholder for the FFP model (assuming fkm is properly defined and used in your environment)
# Initialize FFP model
ffp = fkm.FKM_climatology(domain=[-origin_d, origin_d, -origin_d, origin_d],
                           dx=dx, dy=dx,
                           zm=df4['Zm_2'].tolist(), 
                           z0=df4['z0'].tolist(), 
                           umean=df4['umean'].tolist(),
                           ol=df4['ol'].tolist(), 
                           sigmav=df4['sigma_v'].tolist(), 
                           ustar=df4['u_star'].tolist(), 
                           wind_dir=df4['wind_dir'].tolist())

# Process the footprint data (assuming ffp returns the correct structure)
f_2d = np.array(ffp['fclim_2d']) 
x_2d = np.array(ffp['x_2d']) + station_x
y_2d = np.array(ffp['y_2d']) + station_y
f_2d = f_2d * dx ** 2 

# Create affine transformation for the GeoTIFF
affine_transform = from_origin(station_x - origin_d, station_y + origin_d, dx, dx)

# Set CRS (Coordinate Reference System) using rasterio's CRS
utm_crs = pyproj.CRS.from_epsg(EPSG).to_wkt()
# Prepare to write the GeoTIFF file
out_f = 'new_autmn_5_stable.tif'
with rasterio.open(out_f, 'w', driver='GTiff', dtype=rasterio.float64,
                   count=1,  # Only one band for this footprint
                   height=f_2d.shape[0], width=f_2d.shape[1],
                   transform=affine_transform,
                   crs=utm_crs,
                   nodata=0.0) as new_dat:
    new_dat.write(f_2d, 1)  # Write to the first band

print(f"GeoTIFF saved as {out_f}")

Station coordinates in UTM: X = 292307.3593888789, Y = 2493792.267627137, EPSG = 32643
Percent: [--------------------------------------------------] 0.0% 

d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: divide by zero encountered in divide
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: overflow encountered in exp
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: overflow encountered in multiply
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))


Percent: [################################################--] 96.0% 99999999999%  

45 times Error(0007):
 zm/ol (measurement height to Obukhov length ratio) must be equal or larger than -15.5.
1 times Error(0008):
 sigmav (standard deviation of crosswind) must be larger than zero.
84 times Error(0009):
 ustar (friction velocity) must be >=0.1.


GeoTIFF saved as new_autmn_5_stable.tif


In [8]:
# df4['mm'].unique()

In [17]:
# Constants and parameters
zm_s = 2.83  # Measurement height [m] from AMF metadata
h_s = 2000.  # Height of atmospheric boundary layer [m] - assumed
h_c = np.array(df1['z0'].tolist())  # Height of atmospheric boundary layer [m]
dx = 0.5  # Model resolution [m]
origin_d = 200.  # Model bounds distance from origin [m]

# Station coordinates (lat, long)
latitude = 22.53809845
longitude = 72.98046952
station_coord = (latitude, longitude)

# Calculate the appropriate EPSG code for UTM based on latitude and longitude
EPSG = int(32700 - np.round((45 + latitude) / 90.0) * 100 + np.round((183 + longitude) / 6.0))

# Use pyproj to transform from lat/long (EPSG:4326) to UTM coordinates
transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG}", always_xy=True)
station_x, station_y = transformer.transform(longitude, latitude)

# Check the UTM coordinate results
print(f"Station coordinates in UTM: X = {station_x}, Y = {station_y}, EPSG = {EPSG}")

# Placeholder for the FFP model (assuming fkm is properly defined and used in your environment)
# Initialize FFP model
ffp = fkm.FKM_climatology(domain=[-origin_d, origin_d, -origin_d, origin_d],
                           dx=dx, dy=dx,
                           zm=df1['Zm_1'].tolist(), 
                           z0=df1['z0'].tolist(), 
                           umean=df1['umean1'].tolist(),
                           ol=df1['ol'].tolist(), 
                           sigmav=df1['sigma_v'].tolist(), 
                           ustar=df1['u_star'].tolist(), 
                           wind_dir=df1['wind_dir'].tolist())

# Process the footprint data (assuming ffp returns the correct structure)
f_2d = np.array(ffp['fclim_2d']) 
x_2d = np.array(ffp['x_2d']) + station_x
y_2d = np.array(ffp['y_2d']) + station_y
f_2d = f_2d * dx ** 2 

# Create affine transformation for the GeoTIFF
affine_transform = from_origin(station_x - origin_d, station_y + origin_d, dx, dx)

# Set CRS (Coordinate Reference System) using rasterio's CRS
utm_crs = pyproj.CRS.from_epsg(EPSG).to_wkt()
# Prepare to write the GeoTIFF file
out_f = 'new_winter_2_unstable.tif'
with rasterio.open(out_f, 'w', driver='GTiff', dtype=rasterio.float64,
                   count=1,  # Only one band for this footprint
                   height=f_2d.shape[0], width=f_2d.shape[1],
                   transform=affine_transform,
                   crs=utm_crs,
                   nodata=0.0) as new_dat:
    new_dat.write(f_2d, 1)  # Write to the first band

print(f"GeoTIFF saved as {out_f}")

Station coordinates in UTM: X = 292307.3593888789, Y = 2493792.267627137, EPSG = 32643
Percent: [--------------------------------------------------] 0.0% 

Percent: [#################---------------------------------] 35.0% 

d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:272: RuntimeWarning: invalid value encountered in scalar power
  C = (B**mu) /GammaProxmu
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:273: RuntimeWarning: invalid value encountered in scalar power
  D = sigmav * GammaProx1r / GammaProxmu / ((r * r * Kmaj / Umaj)**(m / r)) / Umaj


Percent: [##################################################] 99.0% 00000000001% 

53 times Error(0007):
 zm/ol (measurement height to Obukhov length ratio) must be equal or larger than -15.5.
2 times Error(0008):
 sigmav (standard deviation of crosswind) must be larger than zero.
18 times Error(0009):
 ustar (friction velocity) must be >=0.1.


GeoTIFF saved as new_winter_2_unstable.tif


In [18]:
# plt.figure(figsize=(14,14))
# src = rasterio.open("winter.tif")
# show(src)

In [19]:
# Constants and parameters
zm_s = 2.83  # Measurement height [m] from AMF metadata
h_s = 2000.  # Height of atmospheric boundary layer [m] - assumed
h_c = np.array(df2['z0'].tolist())  # Height of atmospheric boundary layer [m]
dx = 0.5  # Model resolution [m]
origin_d = 200.  # Model bounds distance from origin [m]

# Station coordinates (lat, long)
latitude = 22.53809845
longitude = 72.98046952
station_coord = (latitude, longitude)

# Calculate the appropriate EPSG code for UTM based on latitude and longitude
EPSG = int(32700 - np.round((45 + latitude) / 90.0) * 100 + np.round((183 + longitude) / 6.0))

# Use pyproj to transform from lat/long (EPSG:4326) to UTM coordinates
transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG}", always_xy=True)
station_x, station_y = transformer.transform(longitude, latitude)

# Check the UTM coordinate results
print(f"Station coordinates in UTM: X = {station_x}, Y = {station_y}, EPSG = {EPSG}")

# Placeholder for the FFP model (assuming fkm is properly defined and used in your environment)
# Initialize FFP model
ffp = fkm.FKM_climatology(domain=[-origin_d, origin_d, -origin_d, origin_d],
                           dx=dx, dy=dx,
                           zm=df2['Zm_1'].tolist(), 
                           z0=df2['z0'].tolist(), 
                           umean=df2['umean1'].tolist(),
                           ol=df2['ol'].tolist(), 
                           sigmav=df2['sigma_v'].tolist(), 
                           ustar=df2['u_star'].tolist(), 
                           wind_dir=df2['wind_dir'].tolist())

# Process the footprint data (assuming ffp returns the correct structure)
f_2d = np.array(ffp['fclim_2d']) 
x_2d = np.array(ffp['x_2d']) + station_x
y_2d = np.array(ffp['y_2d']) + station_y
f_2d = f_2d * dx ** 2 

# Create affine transformation for the GeoTIFF
affine_transform = from_origin(station_x - origin_d, station_y + origin_d, dx, dx)

# Set CRS (Coordinate Reference System) using rasterio's CRS
utm_crs = pyproj.CRS.from_epsg(EPSG).to_wkt()
# Prepare to write the GeoTIFF file
out_f = 'new_Summer_2_unstable.tif'
with rasterio.open(out_f, 'w', driver='GTiff', dtype=rasterio.float64,
                   count=1,  # Only one band for this footprint
                   height=f_2d.shape[0], width=f_2d.shape[1],
                   transform=affine_transform,
                   crs=utm_crs,
                   nodata=0.0) as new_dat:
    new_dat.write(f_2d, 1)  # Write to the first band

print(f"GeoTIFF saved as {out_f}")

Station coordinates in UTM: X = 292307.3593888789, Y = 2493792.267627137, EPSG = 32643
Percent: [--------------------------------------------------] 0.0% 

Percent: [##################################################] 99.0% 00000000001% 

75 times Error(0007):
 zm/ol (measurement height to Obukhov length ratio) must be equal or larger than -15.5.
9 times Error(0008):
 sigmav (standard deviation of crosswind) must be larger than zero.
26 times Error(0009):
 ustar (friction velocity) must be >=0.1.


GeoTIFF saved as new_Summer_2_unstable.tif


In [20]:
# Constants and parameters
zm_s = 2.83  # Measurement height [m] from AMF metadata
h_s = 2000.  # Height of atmospheric boundary layer [m] - assumed
h_c = np.array(df3['z0'].tolist())  # Height of atmospheric boundary layer [m]
dx = 0.5  # Model resolution [m]
origin_d = 200.  # Model bounds distance from origin [m]

# Station coordinates (lat, long)
latitude = 22.53809845
longitude = 72.98046952
station_coord = (latitude, longitude)

# Calculate the appropriate EPSG code for UTM based on latitude and longitude
EPSG = int(32700 - np.round((45 + latitude) / 90.0) * 100 + np.round((183 + longitude) / 6.0))

# Use pyproj to transform from lat/long (EPSG:4326) to UTM coordinates
transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG}", always_xy=True)
station_x, station_y = transformer.transform(longitude, latitude)

# Check the UTM coordinate results
print(f"Station coordinates in UTM: X = {station_x}, Y = {station_y}, EPSG = {EPSG}")

# Placeholder for the FFP model (assuming fkm is properly defined and used in your environment)
# Initialize FFP model
ffp = fkm.FKM_climatology(domain=[-origin_d, origin_d, -origin_d, origin_d],
                           dx=dx, dy=dx,
                           zm=df3['Zm_1'].tolist(), 
                           z0=df3['z0'].tolist(), 
                           umean=df3['umean1'].tolist(),
                           ol=df3['ol'].tolist(), 
                           sigmav=df3['sigma_v'].tolist(), 
                           ustar=df3['u_star'].tolist(), 
                           wind_dir=df3['wind_dir'].tolist())

# Process the footprint data (assuming ffp returns the correct structure)
f_2d = np.array(ffp['fclim_2d']) 
x_2d = np.array(ffp['x_2d']) + station_x
y_2d = np.array(ffp['y_2d']) + station_y
f_2d = f_2d * dx ** 2 

# Create affine transformation for the GeoTIFF
affine_transform = from_origin(station_x - origin_d, station_y + origin_d, dx, dx)

# Set CRS (Coordinate Reference System) using rasterio's CRS
utm_crs = pyproj.CRS.from_epsg(EPSG).to_wkt()
# Prepare to write the GeoTIFF file
out_f = 'new_monsoon_2_unstable.tif'
with rasterio.open(out_f, 'w', driver='GTiff', dtype=rasterio.float64,
                   count=1,  # Only one band for this footprint
                   height=f_2d.shape[0], width=f_2d.shape[1],
                   transform=affine_transform,
                   crs=utm_crs,
                   nodata=0.0) as new_dat:
    new_dat.write(f_2d, 1)  # Write to the first band

print(f"GeoTIFF saved as {out_f}")

Station coordinates in UTM: X = 292307.3593888789, Y = 2493792.267627137, EPSG = 32643
Percent: [--------------------------------------------------] 0.0% 

Percent: [################################################--] 95.0% 00000000001% 

54 times Error(0007):
 zm/ol (measurement height to Obukhov length ratio) must be equal or larger than -15.5.
8 times Error(0008):
 sigmav (standard deviation of crosswind) must be larger than zero.
41 times Error(0009):
 ustar (friction velocity) must be >=0.1.


GeoTIFF saved as new_monsoon_2_unstable.tif


In [21]:
# Constants and parameters
zm_s = 2.83  # Measurement height [m] from AMF metadata
h_s = 2000.  # Height of atmospheric boundary layer [m] - assumed
h_c = np.array(df4['z0'].tolist())  # Height of atmospheric boundary layer [m]
dx = 0.5  # Model resolution [m]
origin_d = 200.  # Model bounds distance from origin [m]

# Station coordinates (lat, long)
latitude = 22.53809845
longitude = 72.98046952
station_coord = (latitude, longitude)

# Calculate the appropriate EPSG code for UTM based on latitude and longitude
EPSG = int(32700 - np.round((45 + latitude) / 90.0) * 100 + np.round((183 + longitude) / 6.0))

# Use pyproj to transform from lat/long (EPSG:4326) to UTM coordinates
transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG}", always_xy=True)
station_x, station_y = transformer.transform(longitude, latitude)

# Check the UTM coordinate results
print(f"Station coordinates in UTM: X = {station_x}, Y = {station_y}, EPSG = {EPSG}")

# Placeholder for the FFP model (assuming fkm is properly defined and used in your environment)
# Initialize FFP model
ffp = fkm.FKM_climatology(domain=[-origin_d, origin_d, -origin_d, origin_d],
                           dx=dx, dy=dx,
                           zm=df4['Zm_1'].tolist(), 
                           z0=df4['z0'].tolist(), 
                           umean=df4['umean1'].tolist(),
                           ol=df4['ol'].tolist(), 
                           sigmav=df4['sigma_v'].tolist(), 
                           ustar=df4['u_star'].tolist(), 
                           wind_dir=df4['wind_dir'].tolist())

# Process the footprint data (assuming ffp returns the correct structure)
f_2d = np.array(ffp['fclim_2d']) 
x_2d = np.array(ffp['x_2d']) + station_x
y_2d = np.array(ffp['y_2d']) + station_y
f_2d = f_2d * dx ** 2 

# Create affine transformation for the GeoTIFF
affine_transform = from_origin(station_x - origin_d, station_y + origin_d, dx, dx)

# Set CRS (Coordinate Reference System) using rasterio's CRS
utm_crs = pyproj.CRS.from_epsg(EPSG).to_wkt()
# Prepare to write the GeoTIFF file
out_f = 'new_autumn_2_unstable.tif'
with rasterio.open(out_f, 'w', driver='GTiff', dtype=rasterio.float64,
                   count=1,  # Only one band for this footprint
                   height=f_2d.shape[0], width=f_2d.shape[1],
                   transform=affine_transform,
                   crs=utm_crs,
                   nodata=0.0) as new_dat:
    new_dat.write(f_2d, 1)  # Write to the first band

print(f"GeoTIFF saved as {out_f}")

Station coordinates in UTM: X = 292307.3593888789, Y = 2493792.267627137, EPSG = 32643
Percent: [--------------------------------------------------] 0.0% 

Percent: [#################################################-] 98.0% 

5 times Error(0007):
 zm/ol (measurement height to Obukhov length ratio) must be equal or larger than -15.5.
2 times Error(0008):
 sigmav (standard deviation of crosswind) must be larger than zero.
10 times Error(0009):
 ustar (friction velocity) must be >=0.1.


GeoTIFF saved as new_autumn_2_unstable.tif


In [13]:
dff = pd.read_csv('unstable.csv')

In [14]:
dff

,mm,day,yyyy,HH_UTC,MM,umean1,umean,wind_dir,z0,ol,u_star,sigma_v,Zm_1,Zm_2
0,1,1,2009,9,4,3.95,4.40,357,0.011764,-105.0200,0.30006,0.17,2.83,5.66
1,1,1,2009,10,2,3.55,4.05,356,0.146380,-6.0125,0.63830,0.42,2.83,5.66
2,1,1,2009,11,1,5.90,6.90,23,0.151490,-22.0110,0.91014,0.25,2.83,5.66
3,1,1,2009,11,5,2.40,2.95,30,0.182490,-85.7220,0.37356,0.13,2.83,5.66
4,1,1,2009,13,0,3.95,4.95,33,0.338130,-23.6200,0.87134,0.57,2.83,5.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3461,12,27,2009,12,2,2.10,2.65,39,0.212590,-705.2400,0.33336,0.63,NaN,NaN
3462,12,28,2009,12,0,1.70,2.70,298,0.917510,-518.5200,0.61677,0.23,NaN,NaN
3463,12,29,2009,11,5,0.40,0.95,358,1.499400,-21.5680,0.48090,0.16,NaN,NaN
3464,12,29,2009,12,2,0.65,1.50,222,1.675400,-45.6680,0.67442,0.15,NaN,NaN


In [9]:
# Constants and parameters
zm_s = 5.66  # Measurement height [m] from AMF metadata
h_s = 2000.  # Height of atmospheric boundary layer [m] - assumed
h_c = np.array(df['z0'].tolist())  # Height of atmospheric boundary layer [m]
dx = 0.5  # Model resolution [m]
origin_d = 200.  # Model bounds distance from origin [m]

# Station coordinates (lat, long)
latitude = 22.53809845
longitude = 72.98046952
station_coord = (latitude, longitude)

# Calculate the appropriate EPSG code for UTM based on latitude and longitude
EPSG = int(32700 - np.round((45 + latitude) / 90.0) * 100 + np.round((183 + longitude) / 6.0))

# Use pyproj to transform from lat/long (EPSG:4326) to UTM coordinates
transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG}", always_xy=True)
station_x, station_y = transformer.transform(longitude, latitude)

# Check the UTM coordinate results
print(f"Station coordinates in UTM: X = {station_x}, Y = {station_y}, EPSG = {EPSG}")

# Placeholder for the FFP model (assuming fkm is properly defined and used in your environment)
# Initialize FFP model
ffp = fkm.FKM_climatology(domain=[-origin_d, origin_d, -origin_d, origin_d],
                           dx=dx, dy=dx,
                           zm=df['Zm_2'].tolist(), 
                           z0=df['z0'].tolist(), 
                           umean=df['umean'].tolist(),
                           ol=df['ol'].tolist(), 
                           sigmav=df['sigma_v'].tolist(), 
                           ustar=df['u_star'].tolist(), 
                           wind_dir=df['wind_dir'].tolist())

# Process the footprint data (assuming ffp returns the correct structure)
f_2d = np.array(ffp['fclim_2d']) 
x_2d = np.array(ffp['x_2d']) + station_x
y_2d = np.array(ffp['y_2d']) + station_y
f_2d = f_2d * dx ** 2 

# Create affine transformation for the GeoTIFF
affine_transform = from_origin(station_x - origin_d, station_y + origin_d, dx, dx)

# Set CRS (Coordinate Reference System) using rasterio's CRS
utm_crs = pyproj.CRS.from_epsg(EPSG).to_wkt()
# Prepare to write the GeoTIFF file
out_f = 'stable_5.tif'
with rasterio.open(out_f, 'w', driver='GTiff', dtype=rasterio.float64,
                   count=1,  # Only one band for this footprint
                   height=f_2d.shape[0], width=f_2d.shape[1],
                   transform=affine_transform,
                   crs=utm_crs,
                   nodata=0.0) as new_dat:
    new_dat.write(f_2d, 1)  # Write to the first band

print(f"GeoTIFF saved as {out_f}")

Station coordinates in UTM: X = 292307.3593888789, Y = 2493792.267627137, EPSG = 32643
Percent: [--------------------------------------------------] 0.0% 

d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: divide by zero encountered in divide
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: overflow encountered in exp
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: overflow encountered in multiply
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))


Percent: [#######################---------------------------] 45.0% 

c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\ma\core.py:1020: RuntimeWarning: overflow encountered in multiply
  result = self.f(da, db, *args, **kwargs)


Percent: [##################################################] 100% Done...00001% 


2 times Error(0003):
 z0 (roughness length) must be larger than zero.
402 times Error(0007):
 zm/ol (measurement height to Obukhov length ratio) must be equal or larger than -15.5.
14 times Error(0008):
 sigmav (standard deviation of crosswind) must be larger than zero.
796 times Error(0009):
 ustar (friction velocity) must be >=0.1.


GeoTIFF saved as stable_5.tif


In [8]:
# Constants and parameters
zm_s = 2.83  # Measurement height [m] from AMF metadata
h_s = 2000.  # Height of atmospheric boundary layer [m] - assumed
h_c = np.array(df['z0'].tolist())  # Height of atmospheric boundary layer [m]
dx = 0.5  # Model resolution [m]
origin_d = 200.  # Model bounds distance from origin [m]

# Station coordinates (lat, long)
latitude = 22.53809845
longitude = 72.98046952
station_coord = (latitude, longitude)

# Calculate the appropriate EPSG code for UTM based on latitude and longitude
EPSG = int(32700 - np.round((45 + latitude) / 90.0) * 100 + np.round((183 + longitude) / 6.0))

# Use pyproj to transform from lat/long (EPSG:4326) to UTM coordinates
transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG}", always_xy=True)
station_x, station_y = transformer.transform(longitude, latitude)

# Check the UTM coordinate results
print(f"Station coordinates in UTM: X = {station_x}, Y = {station_y}, EPSG = {EPSG}")

# Placeholder for the FFP model (assuming fkm is properly defined and used in your environment)
# Initialize FFP model
ffp = fkm.FKM_climatology(domain=[-origin_d, origin_d, -origin_d, origin_d],
                           dx=dx, dy=dx,
                           zm=df['Zm_1'].tolist(), 
                           z0=df['z0'].tolist(), 
                           umean=df['umean1'].tolist(),
                           ol=df['ol'].tolist(), 
                           sigmav=df['sigma_v'].tolist(), 
                           ustar=df['u_star'].tolist(), 
                           wind_dir=df['wind_dir'].tolist())

# Process the footprint data (assuming ffp returns the correct structure)
f_2d = np.array(ffp['fclim_2d']) 
x_2d = np.array(ffp['x_2d']) + station_x
y_2d = np.array(ffp['y_2d']) + station_y
f_2d = f_2d * dx ** 2 

# Create affine transformation for the GeoTIFF
affine_transform = from_origin(station_x - origin_d, station_y + origin_d, dx, dx)

# Set CRS (Coordinate Reference System) using rasterio's CRS
utm_crs = pyproj.CRS.from_epsg(EPSG).to_wkt()
# Prepare to write the GeoTIFF file
out_f = 'stable_2.tif'
with rasterio.open(out_f, 'w', driver='GTiff', dtype=rasterio.float64,
                   count=1,  # Only one band for this footprint
                   height=f_2d.shape[0], width=f_2d.shape[1],
                   transform=affine_transform,
                   crs=utm_crs,
                   nodata=0.0) as new_dat:
    new_dat.write(f_2d, 1)  # Write to the first band

print(f"GeoTIFF saved as {out_f}")

Station coordinates in UTM: X = 292307.3593888789, Y = 2493792.267627137, EPSG = 32643
Percent: [--------------------------------------------------] 0.0% 

d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: divide by zero encountered in divide
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: overflow encountered in exp
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: overflow encountered in multiply
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:272: RuntimeWarning: invalid value encountered in scalar power
  C = (B**mu) /GammaProxmu
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:273: RuntimeWarning: invalid value encountered in scalar power
  D = sigmav * GammaProx1r / GammaProxmu / ((r * r * Kmaj / Umaj)**(m / r)) / Umaj


Percent: [##################################################] 100% Done...00001% 


2 times Error(0003):
 z0 (roughness length) must be larger than zero.
193 times Error(0007):
 zm/ol (measurement height to Obukhov length ratio) must be equal or larger than -15.5.
15 times Error(0008):
 sigmav (standard deviation of crosswind) must be larger than zero.
1002 times Error(0009):
 ustar (friction velocity) must be >=0.1.


GeoTIFF saved as stable_2.tif


In [15]:
# Constants and parameters
zm_s = 5.66  # Measurement height [m] from AMF metadata
h_s = 2000.  # Height of atmospheric boundary layer [m] - assumed
h_c = np.array(dff['z0'].tolist())  # Height of atmospheric boundary layer [m]
dx = 0.5  # Model resolution [m]
origin_d = 200.  # Model bounds distance from origin [m]

# Station coordinates (lat, long)
latitude = 22.53809845
longitude = 72.98046952
station_coord = (latitude, longitude)

# Calculate the appropriate EPSG code for UTM based on latitude and longitude
EPSG = int(32700 - np.round((45 + latitude) / 90.0) * 100 + np.round((183 + longitude) / 6.0))

# Use pyproj to transform from lat/long (EPSG:4326) to UTM coordinates
transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG}", always_xy=True)
station_x, station_y = transformer.transform(longitude, latitude)

# Check the UTM coordinate results
print(f"Station coordinates in UTM: X = {station_x}, Y = {station_y}, EPSG = {EPSG}")

# Placeholder for the FFP model (assuming fkm is properly defined and used in your environment)
# Initialize FFP model
ffp = fkm.FKM_climatology(domain=[-origin_d, origin_d, -origin_d, origin_d],
                           dx=dx, dy=dx,
                           zm=dff['Zm_2'].tolist(), 
                           z0=dff['z0'].tolist(), 
                           umean=dff['umean'].tolist(),
                           ol=dff['ol'].tolist(), 
                           sigmav=dff['sigma_v'].tolist(), 
                           ustar=dff['u_star'].tolist(), 
                           wind_dir=dff['wind_dir'].tolist())

# Process the footprint data (assuming ffp returns the correct structure)
f_2d = np.array(ffp['fclim_2d']) 
x_2d = np.array(ffp['x_2d']) + station_x
y_2d = np.array(ffp['y_2d']) + station_y
f_2d = f_2d * dx ** 2 

# Create affine transformation for the GeoTIFF
affine_transform = from_origin(station_x - origin_d, station_y + origin_d, dx, dx)

# Set CRS (Coordinate Reference System) using rasterio's CRS
utm_crs = pyproj.CRS.from_epsg(EPSG).to_wkt()
# Prepare to write the GeoTIFF file
out_f = 'unstable_5.tif'
with rasterio.open(out_f, 'w', driver='GTiff', dtype=rasterio.float64,
                   count=1,  # Only one band for this footprint
                   height=f_2d.shape[0], width=f_2d.shape[1],
                   transform=affine_transform,
                   crs=utm_crs,
                   nodata=0.0) as new_dat:
    new_dat.write(f_2d, 1)  # Write to the first band

print(f"GeoTIFF saved as {out_f}")

Station coordinates in UTM: X = 292307.3593888789, Y = 2493792.267627137, EPSG = 32643
Percent: [--------------------------------------------------] 0.0% 

d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: divide by zero encountered in divide
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: overflow encountered in exp
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:294: RuntimeWarning: overflow encountered in multiply
  help3 = C*numpy.exp(-B/x_ma)*(x_ma**(-A))


Percent: [##################################################] 100.0% 0000000001% 

478 times Error(0007):
 zm/ol (measurement height to Obukhov length ratio) must be equal or larger than -15.5.
18 times Error(0008):
 sigmav (standard deviation of crosswind) must be larger than zero.
82 times Error(0009):
 ustar (friction velocity) must be >=0.1.


GeoTIFF saved as unstable_5.tif


In [16]:
# Constants and parameters
zm_s = 2.83  # Measurement height [m] from AMF metadata
h_s = 2000.  # Height of atmospheric boundary layer [m] - assumed
h_c = np.array(dff['z0'].tolist())  # Height of atmospheric boundary layer [m]
dx = 0.5  # Model resolution [m]
origin_d = 200.  # Model bounds distance from origin [m]

# Station coordinates (lat, long)
latitude = 22.53809845
longitude = 72.98046952
station_coord = (latitude, longitude)

# Calculate the appropriate EPSG code for UTM based on latitude and longitude
EPSG = int(32700 - np.round((45 + latitude) / 90.0) * 100 + np.round((183 + longitude) / 6.0))

# Use pyproj to transform from lat/long (EPSG:4326) to UTM coordinates
transformer = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{EPSG}", always_xy=True)
station_x, station_y = transformer.transform(longitude, latitude)

# Check the UTM coordinate results
print(f"Station coordinates in UTM: X = {station_x}, Y = {station_y}, EPSG = {EPSG}")

# Placeholder for the FFP model (assuming fkm is properly defined and used in your environment)
# Initialize FFP model
ffp = fkm.FKM_climatology(domain=[-origin_d, origin_d, -origin_d, origin_d],
                           dx=dx, dy=dx,
                           zm=dff['Zm_1'].tolist(), 
                           z0=dff['z0'].tolist(), 
                           umean=dff['umean1'].tolist(),
                           ol=dff['ol'].tolist(), 
                           sigmav=dff['sigma_v'].tolist(), 
                           ustar=dff['u_star'].tolist(), 
                           wind_dir=dff['wind_dir'].tolist())

# Process the footprint data (assuming ffp returns the correct structure)
f_2d = np.array(ffp['fclim_2d']) 
x_2d = np.array(ffp['x_2d']) + station_x
y_2d = np.array(ffp['y_2d']) + station_y
f_2d = f_2d * dx ** 2 

# Create affine transformation for the GeoTIFF
affine_transform = from_origin(station_x - origin_d, station_y + origin_d, dx, dx)

# Set CRS (Coordinate Reference System) using rasterio's CRS
utm_crs = pyproj.CRS.from_epsg(EPSG).to_wkt()
# Prepare to write the GeoTIFF file
out_f = 'unstable_2.tif'
with rasterio.open(out_f, 'w', driver='GTiff', dtype=rasterio.float64,
                   count=1,  # Only one band for this footprint
                   height=f_2d.shape[0], width=f_2d.shape[1],
                   transform=affine_transform,
                   crs=utm_crs,
                   nodata=0.0) as new_dat:
    new_dat.write(f_2d, 1)  # Write to the first band

print(f"GeoTIFF saved as {out_f}")

Station coordinates in UTM: X = 292307.3593888789, Y = 2493792.267627137, EPSG = 32643
Percent: [###-----------------------------------------------] 5.0% 

d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:272: RuntimeWarning: invalid value encountered in scalar power
  C = (B**mu) /GammaProxmu
d:\AAU\Module_3\Project_mo_3\scripts\scripts\footprint_FKM_climatology.py:273: RuntimeWarning: invalid value encountered in scalar power
  D = sigmav * GammaProx1r / GammaProxmu / ((r * r * Kmaj / Umaj)**(m / r)) / Umaj


Percent: [##################################################] 100.0% 0000000001% 

187 times Error(0007):
 zm/ol (measurement height to Obukhov length ratio) must be equal or larger than -15.5.
21 times Error(0008):
 sigmav (standard deviation of crosswind) must be larger than zero.
95 times Error(0009):
 ustar (friction velocity) must be >=0.1.


GeoTIFF saved as unstable_2.tif
